[MIPROv2](https://dspy.ai/api/optimizers/MIPROv2/)

At a high level, MIPROv2 works by creating both few-shot examples and new instructions for each predictor in your LM program, and then searching over these using Bayesian Optimization to find the best combination of these variables for your program. If you want a visual explanation check out this [twitter thread](https://x.com/michaelryan207/status/1804189184988713065)

In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5500")
mlflow.set_experiment("optimize-react")
mlflow.dspy.autolog(
    log_compiles=True,    # Track optimization process
    log_evals=True,       # Track evaluation results
    log_traces_from_compile=True  # Track program traces during optimization
)

/home/mike/git/dspy-tool-use/venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import dspy
import openai
import os

LLM_URL=os.getenv('LLM_URL', 'http://localhost:8080/v1')
API_KEY=os.getenv('API_KEY', 'fake')
LLM_MODEL=os.getenv('LLM_MODEL', 'openai/models/Llama-3.2-3B-Instruct-Q8_0.gguf')
MAX_TOKENS=os.getenv('MAX_TOKENS', 3000)
TEMPERATURE=os.getenv('TEMPERATURE', 0.2)
dspy.enable_logging()
lm = dspy.LM(model=LLM_MODEL,
             api_base=LLM_URL,  # ensure this points to your port
             api_key=API_KEY,
             temperature=TEMPERATURE,
             model_type='chat',
             stream=False)
dspy.configure(lm=lm)
dspy.settings.configure(track_usage=True)

In [ ]:
from dspy.datasets import HotPotQA

def search(query: str) -> list[str]:
    """Retrieves abstracts from Wikipedia."""
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

trainset = [x.with_inputs('question') for x in HotPotQA(train_seed=2024, train_size=20).train]
react = dspy.ReAct("question -> answer", tools=[search])

tp = dspy.MIPROv2(metric=dspy.evaluate.answer_exact_match, auto=None, num_threads=24, num_candidates=10)  # auto="light"
optimized_react = tp.compile(react, trainset=trainset, requires_permission_to_run=False, num_trials=2, minibatch=True, minibatch_size=10, minibatch_full_eval_steps=2)

optimized_react.save(path="optimized_react.json")

Using the latest cached version of the module from /home/mike/.cache/huggingface/modules/datasets_modules/datasets/hotpot_qa/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5 (last modified on Sat Aug 17 10:45:14 2024) since it couldn't be found locally at hotpot_qa, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/mike/.cache/huggingface/modules/datasets_modules/datasets/hotpot_qa/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5 (last modified on Sat Aug 17 10:45:14 2024) since it couldn't be found locally at hotpot_qa, or remotely on the Hugging Face Hub.
2025/06/03 10:06:19 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6308fe4da04040d1821126afa9d02c37', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/06/03 10:06:19 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <

Bootstrapping set 1/10
Bootstrapping set 2/10
Bootstrapping set 3/10


100%|██████████| 4/4 [00:00<00:00, 16.82it/s]


Bootstrapped 1 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.


Bootstrapping set 4/10


...

Average Metric: 3.00 / 16 (18.8%): 100%|██████████| 16/16 [01:54<00:00,  7.19s/it]

2025/06/03 10:22:31 INFO dspy.evaluate.evaluate: Average Metric: 3 / 16 (18.8%)
2025/06/03 10:22:31 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 18.75
2025/06/03 10:22:31 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [0.0, 18.75]
2025/06/03 10:22:31 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 18.75
2025/06/03 10:22:31 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/06/03 10:22:31 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/06/03 10:22:31 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 18.75!



🏃 View run eval_full_1 at: http://localhost:5500/#/experiments/302620010842219766/runs/8a401584b82a4938a9ae34281dc69924
🧪 View experiment at: http://localhost:5500/#/experiments/302620010842219766


🏃 View run delightful-stoat-295 at: http://localhost:5500/#/experiments/302620010842219766/runs/6308fe4da04040d1821126afa9d02c37
🧪 View experiment at: http://localhost:5500/#/experiments/302620010842219766


[Trace(request_id=ebf6c00c07334097861866a7fe14584c), Trace(request_id=97a56318c9094a7c9a342dd9af2602b6), Trace(request_id=d1304a8ce5f54e77a67c470c8b59080d), Trace(request_id=8071a6349d0f478495c21fcacf9ce07e), Trace(request_id=c020be3512484a84a9df60520b2b73d8), Trace(request_id=a5715b8b4f9d4a60852d99c869285e43), Trace(request_id=d8e8f2f792224b5ab015d82483cdfb43), Trace(request_id=3087a280626043fdba8cb3cc49fb5529), Trace(request_id=f7c5240269474f93a655a3c35a6faa3b), Trace(request_id=3cdf50c110f54b0d92c4fa950fac974f)]

In [4]:
loaded_optimize_react = dspy.ReAct("question -> answer", tools=[search])
loaded_optimize_react.load("optimized_react.json")

loaded_optimize_react(question="Is Liverpool found in northern or southern England?")

Prediction(
    trajectory={'thought_0': 'Liverpool is located in the north of England.', 'tool_name_0': 'search', 'tool_args_0': {'query': 'Is Liverpool in northern England?'}, 'observation_0': ['Liverpool | Liverpool ( ) is a city in North West England, with an estimated population of 478,580 in 2015. With its surrounding areas, it is the fifth-largest metropolitan area in the UK, with over 2.24 million people in 2011. The local authority is Liverpool City Council, the most populous local government district within the metropolitan county of Merseyside and the largest within the Liverpool City Region.', 'Demography of Liverpool | The demography of Liverpool is officially analysed by the Office for National Statistics. The City of Liverpool together with the Metropolitan Borough of Sefton, the Metropolitan Borough of Knowsley, the Metropolitan Borough of St Helens and the Metropolitan Borough of Wirral forms the metropolitan county of Merseyside. With a population of around 466,400, L

Trace(request_id=5c3423a631a342bbadb47132e1fa360e)